# 加速：多線程爬蟲



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭

## 作業目標

* 找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。





In [6]:
# 打開瀏覽器

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def get_SO2(site):
    browser = webdriver.Chrome(executable_path='chromedriver')

    browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

    # 模擬使用者操作行為，選擇/點擊

    selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
    selectSite.select_by_value(site)
    selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
    selectYear.select_by_value('2018')

    browser.find_element_by_id('ctl05_btnQuery').click()

    locator=("id", "ctl05_gv")
    try:
        WebDriverWait(browser, 30, 0.5).until(EC.presence_of_element_located(locator))
    finally:
        print('')

    # 取得資料，丟到 BeautifulSoup 解析

    html_source = browser.page_source

    soup = BeautifulSoup(html_source, 'html.parser')
    table = soup.find('table', class_='TABLE_G')
    trs=table.find_all('tr')

    bool_so2=False
    str_date=''
    str_value=''
    for tr in trs:
        tds=tr.find_all('td')
        if (len(tds)>0):
            if (len(tds)==5):
                bool_so2=tds[0].text=='SO2'
                if (bool_so2):
                    str_date=tds[2].text
                    str_value=tds[3].text
            else:
                str_date=tds[0].text
                str_value=tds[1].text
        if (bool_so2):
            if (str_date<='2018/08'):
                print(f"{str_date} / {str_value}")

In [7]:
import time

startTime = time.time()
get_SO2('11')
get_SO2('6')
finishTime = time.time()
print(finishTime - startTime) # 正常情況的爬蟲所需時間


2018/01 / 1.80
2018/02 / 1.90
2018/03 / 2.20
2018/04 / 2.30
2018/05 / 3.10
2018/06 / 2.70
2018/07 / 2.20
2018/08 / 2.40

2018/01 / 2.40
2018/02 / 2.60
2018/03 / 2.90
2018/04 / 3.20
2018/05 / 4.20
2018/06 / 3
2018/07 / 3.10
2018/08 / 3.90
18.33515167236328


In [10]:
import _thread

sites = ['11', '6']

startTime = time.time()

for site in sites:
    _thread.start_new_thread( get_SO2, (site,) )

finishTime = time.time()
print(finishTime - startTime) # 利用 thread 的爬蟲所需時間

0.0

2018/01 / 2.40
2018/02 / 2.60
2018/03 / 2.90
2018/04 / 3.20
2018/05 / 4.20
2018/06 / 3
2018/07 / 3.10
2018/08 / 3.90

2018/01 / 1.80
2018/02 / 1.90
2018/03 / 2.20
2018/04 / 2.30
2018/05 / 3.10
2018/06 / 2.70
2018/07 / 2.20
2018/08 / 2.40
